# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
from IPython import display
import warnings

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download(['wordnet', 'punkt', 'stopwords'])

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, BaseEstimator, TfidfTransformer
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, accuracy_score


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM InsertTableName;", engine)
X = df.message.values
Y = df.iloc[:, 4:].values
category_label = df.iloc[:, 4:].columns.tolist()
print(category_label)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [3]:
def silence_print(true_label, pred_label, silence=True):
    """Print classification report without warning message
    
    Parameters:
    -----------
    true_label: array
        The true values
    pred_label: array
        The predict values
    silence: boolean default True
        If true, print the report without warning message
    """
    if silence:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            for index, column in enumerate(category_label):
                report = classification_report(
                    true_label[:, index], pred_label[:, index], labels=np.arange(1), \
                    target_names=[column], digits=2               
                )
                
                print("{0:<25}\n  Accuracy:{1: 5.3f}  Precision:{2: =6.3f}   Recall:{3:< 5.3f}".format(
                    column, accuracy_score(true_label[:, index], pred_label[:, index]), 
                    report[column]["precision"], report[column]["recall"], report[column]["f1-score"]
                ))
    else:
        for index, column in enumerate(category_label):
            report = classification_report(
                    true_label[:, index], pred_label[:, index], labels=np.arange(1), \
                    target_names=[column], digits=2
                )
            print("{0:<25}\n  Accuracy:{1: 5.3f}  Precision:{2: =6.3f}   Recall:{3:< 5.3f}".format(
                    column, accuracy_score(true_label[:, index], pred_label[:, index]), 
                    report[column]["precision"], report[column]["recall"], report[column]["f1-score"]
                ))

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """Tokenize the text 
    
    Tokenize the text information by using word_tokenize, WordNetLemmatizer, 
    remove words that is in stopwords
    
    Parameters:
    -----------
    text: string
        String contains the message information
    
    Results:
    ----------
    result: list
        List contains strings parsed from the text
    """
    # nromalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    result = []
    for token in tokens:
        if token not in stop_words:
            result.append(lemmatizer.lemmatize(token))
    return result


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
forest = RandomForestClassifier(random_state=42, n_jobs=4) 


pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
    ])),
    ('clf', MultiOutputClassifier(forest, n_jobs=2))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#spliting data
train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size=0.3, random_state=42)

In [7]:
#trainig data

pipeline.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=2))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# basic model

pred_y = pipeline.predict(test_X)

In [9]:
# print the report
import sklearn
print(sklearn.__version__)
silence_print(pred_y, test_y)

0.19.1


TypeError: string indices must be integers

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
forest = RandomForestClassifier(random_state=42) 


pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
    ])),
    ('clf', MultiOutputClassifier(forest))
])

In [ ]:
# params
params = {
    'text_pipeline__vect__max_features': (5000, 100000),
    'clf__estimator__n_estimators': [10, 20, 30],
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_depth': [4, 6, 10],

}

In [ ]:
cv = GridSearchCV(pipeline, param_grid=params, cv=3)

cv.fit(train_X, train_y)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# predicting values
pred_y = cv.predict(test_X)

In [ ]:
# print report

silence_print(test_y, pred_y)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
def silence_print(true_label, pred_label, silence=True):
    """Print classification report without warning message
    
    Parameters:
    -----------
    true_label: array
        The true values
    pred_label: array
        The predict values
    silence: boolean default True
        If true, print the report without warning message
    """
    if silence:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            for index, column in enumerate(category_label):
                report = classification_report(
                    true_label[:, index], pred_label[:, index], labels=np.arange(1), \
                    target_names=[column], digits=2
                )
                print("{0:<25}\n  Accuracy:{1: 5.3f}  Precision:{2: =6.3f}   Recall:{3:< 5.3f}".format(
                    column, accuracy_score(true_label[:, index], pred_label[:, index]), 
                    0, 1, 2
                ))
    else:
        for index, column in enumerate(category_label):
            report = classification_report(
                    true_label[:, index], pred_label[:, index], labels=np.arange(1), \
                    target_names=[column], digits=2
                )
            print("{0:<25}\n  Accuracy:{1: 5.3f}  Precision:{2: =6.3f}   Recall:{3:< 5.3f}".format(
                    column, accuracy_score(true_label[:, index], pred_label[:, index]), 
                    0, 1, 2
                ))

In [ ]:
def check(x):
    """Parse the text into more information
    Get the more information to analyze, like sentence length over 1
    and check the pos tag type
    
    Parameters:
    -----------
    x: string
        Messages string
    
    Results:
    ----------
    length: boolean
        Check the sentence length is over 1
    chunck: boolean
        Check whether there is specific pos tag
    """
    length = len(sent_tokenize(x))
    chunck = False

    for sentence in sent_tokenize(x):
        pos_tags = nltk.pos_tag(chunck_tokenize(sentence))

        try:
            word, tag = pos_tags[0]

            if "VB" in tag:
                chunck = True
        except:
            continue
        
        try:
            for word, tag in nltk.pos_tag(chunck_tokenize(sentence))[:3]:
                if tag == "NNP":
                    chunck = True
                    break
        except:
            continue
            
        if chunck:
            break

    return (length, chunck)

In [ ]:
class WordChunckExtract(BaseEstimator, TransformerMixin):
    """Word Pos Tag Extract
    
    Word Chunck extract the text information, which is used to 
    fit and transform method
    """
    def starting_verb(self, text):
        """Parse the text into more information
        Get the more information to analyze, like sentence length over 1
        and check the pos tag type

        Parameters:
        -----------
        text: string
            Messages string

        Results:
        ----------
        length: boolean
            Check the sentence length is over 1
        chunck: boolean
            Check whether there is specific pos tag
        """
        length = len(sent_tokenize(text))
        chunck = False

        for sentence in sent_tokenize(text):
            pos_tags = nltk.pos_tag(chunck_tokenize(sentence))

            try:
                word, tag = pos_tags[0]

                if "VB" in tag:
                    chunck = True
            except:
                continue

            try:
                for word, tag in nltk.pos_tag(chunck_tokenize(sentence))[:3]:
                    if tag == "NNP":
                        chunck = True
                        break
            except:
                continue

            if chunck:
                break

        return (length, chunck)

    def fit(self, x, y=None):
        """
        Fit the text message
        """
        return self

    def transform(self, X):
        """
        Transform the text message
        """
        result = pd.DataFrame()
        X_tagged = pd.Series(X).apply(self.starting_verb)
        result["length"] = X_tagged.apply(lambda x: x[0] > 1)
        result["chucnk"] = X_tagged.apply(lambda x: x[1])
        return result

In [ ]:
forest = RandomForestClassifier(random_state=42, n_jobs=4)

pipeline = Pipeline([
    ("features", FeatureUnion([
        ("text_pipeline", Pipeline([
            ("vect", CountVectorizer(tokenizer=tokenize)),
            ("tfidf", TfidfTransformer())
        ])), 
        ("exact_chunck", WordChunckExtract())
    ])),
    ("clf", MultiOutputClassifier(forest, n_jobs=4))
])

In [ ]:
# params
params = {
#     'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#     'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000, 10000),
#     'features__text_pipeline__tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [100, 150],
    'clf__estimator__criterion': ['gini', 'entropy'],
    'clf__estimator__max_depth': [12, 18, 25],
    'features__transformer_weights': (
    {'text_pipeline': 1, 'starting_verb': 0.5},
#     {'text_pipeline': 0.5, 'starting_verb': 1},
#     {'text_pipeline': 0.8, 'starting_verb': 1},
)

}

# train model
cv = GridSearchCV(pipeline, param_grid=params, cv=3)

cv.fit(train_X, train_y)

In [ ]:
# predict values
pred_y = cv.predict(test_X)

In [ ]:
silence_print(test_y, pred_y)

### 9. Export your model as a pickle file

In [ ]:
import pickle

In [ ]:
with open("model.pickle", "wb") as file:
    pickle.dump(cv, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.